In [1]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random
from collections import deque
from sys import maxsize

In [2]:
def createList(r1, r2):#option creation for random edge addition
        if (r1 == r2):
            return r1
        else:
            res = []
            while(r1 != r2):
                res.append(r1)
                r1 += 1
                if (r1 > 50):
                    r1 = r1-50
            res.append(r2)
        return res 

In [3]:
def find_paths(paths, path, parent, end):
    if end == -1:
        paths.append(path.copy())
        return
    
    for par in parent[end]:
        path.append(end)
        find_paths(paths, path, parent, par)
        path.pop()
        
def bfs_all_paths(parent, start, g):
    
    dist = [maxsize for _ in range(51)]
    q = deque()
    q.append(start)
    parent[start] = [-1]
    dist[start] = 0

    while q:
        u = q[0]
        q.popleft()
        for v in g.graph[u]:
            if dist[v] > dist[u] + 1:
                dist[v] = dist[u] + 1
                q.append(v)
                parent[v].clear()
                parent[v].append(u)
                
            elif dist[v] == dist[u] + 1:
                parent[v].append(u)
    return parent

def all_shortest_paths(g):
    shortest_paths = dict()
    for i in range(1, 51):
        for j in range(1, 51):
            paths, path, shortest_path = [], [], []
            
            parent = [[] for _ in range(51)]
            parent = bfs_all_paths(parent, i, g)
            
            paths = find_paths(paths, path, parent, j)
            
            for v in parent:
                v = list(reversed(v))
                shortest_path.append(v)
                
            shortest_paths[i,j] = shortest_path
    
    return shortest_paths


In [4]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
        self.paths = all_shortest_paths(self)
                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1): #deleting adjacent options so that double edges arent made
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else: 
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [5]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [6]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
    
    def pred_distracted_move(self, g, agentpos): #predator moving with probability 0.6 to node closer to agent and probability 0.4 to any other random neighbour
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths 
        new_path = random.randint(0,(len(shortest_path)-1))
        chance = random.randint(1,100)
        if chance <= 60: #60 percent chance of moving correctly
            self.pos = shortest_path[new_path][0]
        else: #40 percent chance of moving in distracted manner
            self.pos = random.choice(neighbors)
        return self
    
    def probability_distracted(self, g, agentpos):
        shortest_paths = g.paths
        count = 0
        for path in shortest_paths:
            if self.pos in path:
                count += 1
        return (0.6 * (count/len(shortest_paths)))
        

In [7]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [10]:
class Utility:
    def __init__(self):
        self.Ustar = np.zeros((51, 51, 51)) # Initialize UStar map
    
    def action_value(self, g, agentpos, preyneighbors, predneighbors, predator):
        future_utility = 0
        for preypos in preyneighbors:
            for predpos in predneighbors:
                # figure out distracted probability
                probability = (1.0/len(preyneighbors))*((0.4/len(predneighbors)) + predator.probability_distracted(g, agentpos))
                future_utility += probability*(1 + self.Ustar[agentpos][preypos][predpos])
        return future_utility

    def value_iteration(self, g, predator):
        
        #initialize UStar
        for agentpos in range(1, 51): 
              for preypos in range(1, 51):
                for predpos in range(1, 51):
                    if (agentpos == preypos) and (agentpos != predpos):
                        self.Ustar[agentpos][preypos][predpos] = 0
                    elif (agentpos == predpos) or (abs(agentpos - predpos) == 1):
                        self.Ustar[agentpos][preypos][predpos] = 1000
                    else:
                        self.Ustar[agentpos][preypos][predpos] = 1

        #Update UStar based on the likely movements of the prey and predator
        while True:
            currUstar = np.zeros((51, 51, 51))
            for agentpos in range(1, 51): 
                for preypos in range(1, 51):
                    for predpos in range(1, 51):
                        if (agentpos == preypos) and (agentpos != predpos):
                            currUstar[agentpos][preypos][predpos] = 0
                        elif (agentpos == predpos) or (abs(agentpos - predpos) == 1):
                            currUstar[agentpos][preypos][predpos] = 1000
                        elif (abs(agentpos - preypos) == 1):
                            currUstar[agentpos][preypos][predpos] = 1
                        else:
                            action_values = []
                            for move in g.graph[agentpos]:
                                # Run loop until value converges
                                preyneighbors = []
                                predneighbors = []
                                for neighbor in g.graph[preypos]:
                                    preyneighbors.append(neighbor)
                                for neighbor in g.graph[predpos]:
                                    predneighbors.append(neighbor)
                                preyneighbors.append(preypos)
                            currval = self.action_value(g, move, preyneighbors, predneighbors, predator)
                            action_values.append(currval)
                            
            #print("After updating UStar")
            currUstar[agentpos][preypos][predpos] = min(action_values) 
      

            max_error = 0
            for agentpos in range(1, 51): 
                for preypos in range(1, 51):
                    for predpos in range(1, 51):
                        max_error = max(max_error, abs(self.Ustar[agentpos][preypos][predpos] - currUstar[agentpos][preypos][predpos]))
            print('error', max_error)            
            if max_error < 0.001:
                break
            self.Ustar = currUstar
        

In [ ]:
g = Graph()
print(g.graph)
print()

graph = g.addEdge()
while graph:
    graph = g.addEdge()
    
print(g.graph)
print()

print("Predator")
p = Predator(1)
print(p.pos)
p = p.pred_distracted_move(g, 1)
print(p.pos)

print("Prey")
prey = Prey(1)
print(prey.pos)
prey = prey.prey_move(g)
print(prey.pos)

utility = Utility()  
utility.value_iteration(g, p)

{1: [2, 50], 2: [3, 1], 3: [4, 2], 4: [5, 3], 5: [6, 4], 6: [7, 5], 7: [8, 6], 8: [9, 7], 9: [10, 8], 10: [11, 9], 11: [12, 10], 12: [13, 11], 13: [14, 12], 14: [15, 13], 15: [16, 14], 16: [17, 15], 17: [18, 16], 18: [19, 17], 19: [20, 18], 20: [21, 19], 21: [22, 20], 22: [23, 21], 23: [24, 22], 24: [25, 23], 25: [26, 24], 26: [27, 25], 27: [28, 26], 28: [29, 27], 29: [30, 28], 30: [31, 29], 31: [32, 30], 32: [33, 31], 33: [34, 32], 34: [35, 33], 35: [36, 34], 36: [37, 35], 37: [38, 36], 38: [39, 37], 39: [40, 38], 40: [41, 39], 41: [42, 40], 42: [43, 41], 43: [44, 42], 44: [45, 43], 45: [46, 44], 46: [47, 45], 47: [48, 46], 48: [49, 47], 49: [50, 48], 50: [1, 49]}

{1: [2, 50, 47], 2: [3, 1, 48], 3: [4, 2, 7], 4: [5, 3, 50], 5: [6, 4, 10], 6: [7, 5, 8], 7: [8, 6, 3], 8: [9, 7, 6], 9: [10, 8, 12], 10: [11, 9, 5], 11: [12, 10], 12: [13, 11, 9], 13: [14, 12, 15], 14: [15, 13, 16], 15: [16, 14, 13], 16: [17, 15, 14], 17: [18, 16, 19], 18: [19, 17, 21], 19: [20, 18, 17], 20: [21, 19, 22], 

In [ ]:
def AgentUStar(g):
    agent_pos = random.randint(1,50)
    predator = Predator(agent_pos)
    prey = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    utility = Utility()
    utility.value_iteration(g, predator)
    
    while (fail == 0 and steps <= 100):
        
        best_node = agent_pos
        min_utility = 10000
        
        for action in g.graph[agent_pos]:
            if utility[action][prey.pos][predator.pos] < min_utility:
                min_utility = utility[action][prey.pos][predator.pos]
                best_node = action
        
        agent_pos = best_node
        path.append(agent_pos)
        steps += 1
        
        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move predator and prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator.pred_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
    #fail if timeout error    
    if(steps >= 100):
        fail = 2 

    return fail, path

In [ ]:
wins = 0
losses = 0
timeout = 0
times = 0

while times < 1:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = AgentUStar(g)
    
    if(res == 0):
        wins += 1
    elif(res == 1):
        losses += 1
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print('Number of losses: ' + str(losses))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

In [ ]:
def AgentUStarPartial(g):
    agent_pos = random.randint(1,50)
    predator = Predator(agent_pos)
    prey = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    utility = Utility()
    utility.value_iteration(g, predator)
    
    while (fail == 0 and steps <= 100):
        
        best_node = agent_pos
        min_utility = 10000
        
        for action in g.graph[agent_pos]:
            if utility[action][prey.pos][predator.pos] < min_utility:
                min_utility = utility[action][prey.pos][predator.pos]
                best_node = action
        
        agent_pos = best_node
        path.append(agent_pos)
        steps += 1
        
        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move predator and prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator.pred_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
    #fail if timeout error    
    if(steps >= 100):
        fail = 2 

    return fail, path

In [ ]:
wins = 0
losses = 0
timeout = 0
times = 0

while times < 1:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = AgentUStarPartial(g)
    
    if(res == 0):
        wins += 1
    elif(res == 1):
        losses += 1
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print('Number of losses: ' + str(losses))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))